# What's New in FEFLOW 7.5 IFM API ?

*contact: mike.de@dhigroup.com*

With FEFLOW 7.5, we have added a number of additional features to FEFLOWS Python Programming interface IFM:

+ Access to the **Lookup Table**
+ Setting **Solver Type**
+ Setting **Number of Threads** for parallel computation

This document shows examples to get you up to speed.

*Note that we are using functionality of the `ifm_contrib` project in some cases, to save unnecessary conde lines and to keep this document concise. `ifm_contrib` is a community project that extents the `ifm` library with additional functionality. You can find more information [here](github.com/dhi/ifm_contrib).*

In [1]:
import os
import sys
import pandas as pd

import matplotlib.pyplot as plt

os.environ["FEFLOW_KERNEL_VERSION"] = "75"  # use the FEFLOW 7.4 API
import ifm_contrib as ifm  # ifm_contrib is code compatible with ifm.

print(f"This is FEFLOW {ifm.getKernelVersion()/1000} (build {ifm.getKernelRevision()}).")

This is FEFLOW 7.458 (build 19494).


In [2]:
# load an example
ifm.forceLicense("Viewer")
doc = ifm.loadDocument("../data/unittests/models/example_2D.fem")

## Lookup Tables

FEFLOW 7.5 introduces lookup tables that allows the assignment of material parameters to elements that have previously been defined to an abstract material type.

**Access the LookupTable**

To access the Lookuptable, run `doc.LookupTable()` to retrieve the table as a Python object.



In [3]:
lookup = doc.getLookupTable()
# note that the table is empty at this point

**Adding and Removing Materials**

The list of materials can be edited using the following functions

In [5]:
# create new materials
lookup.addMaterial("Sand")
lookup.addMaterial("Clay")
lookup.addMaterial("Bedrock")

3

Note that the function returns the Material ID of the material just created. Material ID starts with 1 for consistency with the GUI.


In [6]:
# you can get the material ID from a single existing material ..
mat_id = lookup.findMaterial("Sand")
print(mat_id)

1


In [7]:
# ... and vice versa with
lookup.getMaterialName(1)

'Sand'

In [8]:
# alternatively, this gives you a dictionary of all materials 
# in one shot:
lookup.getMaterials()

{'Sand': 1, 'Clay': 2, 'Bedrock': 3}

In [10]:
# Material IDs can be arbitrarily set
# example: Move 'Sand' to first position (=zero)  
lookup.moveMaterial(lookup.findMaterial("Sand"), 0)

lookup.getMaterials()

{'Sand': 1, 'Clay': 2, 'Bedrock': 3}

In [11]:
# assign a different ID
lookup.setMaterialId(lookup.findMaterial("Sand"), 10)

lookup.getMaterials()

{'Sand': 10, 'Clay': 2, 'Bedrock': 3}

In [12]:
# rename a material
lookup.setMaterialName(lookup.findMaterial("Bedrock"), "Granite")

lookup.getMaterials()

{'Sand': 10, 'Clay': 2, 'Granite': 3}

In [13]:
# finally removes a material
lookup.removeMaterial(lookup.findMaterial("Granite"))

lookup.getMaterials()

{'Sand': 10, 'Clay': 2}

**Changing Properties**

In [14]:
# a property can be added using the material id and parameter id, 
# as well es the value to be set
lookup.setPropertyValue(lookup.findMaterial("Clay"),
                            ifm.Enum.P_COND,
                            1e-6)

lookup.setPropertyValue(lookup.findMaterial("Sand"),
                            ifm.Enum.P_COND,
                            1e-4)

lookup.setPropertyValue(lookup.findMaterial("Sand"),
                            ifm.Enum.P_PORO,
                            1e-4)

In [15]:
# the value can then be retrieved 
lookup.getPropertyValue(lookup.findMaterial("Clay"), 
                        ifm.Enum.P_COND)

1e-06

In [16]:
# this would usually be preceeded by a check if the material 
# is actually set
lookup.hasProperty(lookup.findMaterial("Clay"), 
                        ifm.Enum.P_COND)

True

In [17]:
# alternatively, you can retrieve the complete table in one go
lookup.getProperties(lookup.findMaterial("Clay"))

{100: 1e-06}

In [18]:
# finally, a property can be removed
lookup.removeProperty(lookup.findMaterial("Clay"), 
                        ifm.Enum.P_COND)

In [20]:
lookup.getProperties(lookup.findMaterial("Clay"))

{}

**Assigning Time Series**

Lookup tables allow to assign a time series to a material. (Note: that this is different from assigning time-varying material properties).





In [21]:
lookup.setPropertyValue()

TypeError: syntax: void lookup.setPropertyValue(int material_id, int param_id, object value)

**Working with Pandas**

For users working with the `pandas` Python library, the `ifm_contrib` extension can return the lookup table as a DataFrame:

In [22]:
doc.c.settings.df.lookup_table()

,material_id,100,201
material_name,,,
Sand,10,0.0001,0.0001
Clay,2,NaN,NaN


In [23]:
doc.c.settings.df.lookup_table(names_as_index=False)

,material_name,100,201
material_id,,,
10,Sand,0.0001,0.0001
2,Clay,NaN,NaN


## Computation Settings

### Solver Type

From FEFLOW 7.5, it is possible to choose the most relevant solver option via the API:

+ PARDISP (`pardiso`)
+ PCG (`pcg`)
+ SAMG (Algebraic Multigrid Sovler by Fraunhofer SCAI) (`samg`)
+ PETSc Krylov Subspace Solver (`pksp` or `petsc_ksp`)
+ PETSc Algebraic Multigrid Solver (`pamg` or `petsc_amg`)   

The Solvers from the PETSc library are introduced with FEFLOW 7.5 release.



In [24]:
doc.setSolver("petsc_amg")

### Thread Number

From FEFLOW 7.5 onwards, it is possible to set the thread number for parallel computation.